# Understanding the performance of the vecadd kernel


In this section we will see how we can avoid the JIT cost of Numba, how we can measure the performance of the kernel without the `%timeit` magic, how we can use `nsys profile`, the CUDA profiler to analyze the performance of the kernel, and finally, we will evaluate the performance of the kernel.

## Avoiding the JIT cost

The previous exercise has shown that Numba will compile the CUDA kernel every time we call our program and, in order to amortize the compilation cost, we need several invocations. We would like to avoid this cost.

Unlike the `@numba.jit` decorator, `@cuda.jit` does not accept a `cache` parameter, that would cache the generated code on the disk and use it on subsequent invocations of the program. Nonetheless, we can force the code generation at import time by supplying a function signature to the `@cuda.jit` decorator that describes the CUDA kernel. This will generate the CUDA code at the time when the decorator processes the function declaration and, therefore, we will avoid the runtime cost of JIT. Let's see how this is done:

In [ ]:
import numba.cuda as cuda
import os

import numpy as np
import numba


#@cuda.jit('void(Array(float64, 1, "C"), Array(float64, 1, "C"), Array(float64, 1, "C"))')
@cuda.jit('void(float64[::1], float64[::1], float64[::1])')
def _vecadd_cuda(z, x, y):
    '''The CUDA kernel'''
    i = cuda.grid(1)
    N = x.shape[0]
    if i >= N:
        return

    z[i] = x[i] + y[i]

This instructs the Numba runtime to compile the following function into a CUDA kernel (return type `void`) accepting three one-dimensional arrays of `float64` (or `double`) stored in row-major order (C convention). This way, Numba does not have to wait until the `_vecadd_cuda` function is called to figure out the argument types and compile the kernel. It can do this at import time, when it first encounters the function. The downside to that is that you can't call the function with a different type of arguments later. For more details on how you can specify function signatures in Numba, see [here](https://numba.readthedocs.io/en/latest/reference/types.html#numba-types).

Let's retry our example now with this version of the kernel.

In [ ]:
# Initialize the random engine
rng = np.random.default_rng()

In [ ]:
# Set up the host vectors
N = 200*1000*1000
x = rng.uniform(size=N)
y = rng.uniform(size=N)

# Copy and allocate data on the device
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_z = cuda.device_array_like(x)

# Set up the kernel invocation
block_size = 128
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

# Call the kernel
%timeit -r2 -n 4 _vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)

# Copy back the result to the host
res = d_z.copy_to_host()

# Validate the result
assert np.allclose(x + y, res)

### Exercise

> Time the kernel with `%timeit -n1 -r1`. Try to increase the repetitions and experiment with different array sizes. What do you see?

## Measuring the execution time of the kernel

All you see from the previous exercise is the same execution time! What is happening? Actually, you are not measuring the kernel execution time, but rather the kernel launch time. CUDA kernels are launched asynchronously. This means that as soon as you launch the kernel on the GPU, the CPU will continue execution. In this case, it will continue executing and it will block at the statement that copies back the result to the host. 

How do we measure the kernel execution time then? For this, we are going to write a Python [context manager](https://docs.python.org/3.8/reference/datamodel.html?highlight=__getitem__#with-statement-context-managers) so as to measure the execution time of a region in a nice, Pythonic way:

In [ ]:
import time


class time_region:
    def __init__(self, time_offset=0):
        self._time_off = time_offset

    def __enter__(self):
        self._t_start = time.time()
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self._t_end = time.time()

    def elapsed_time(self):
        return self._time_off + (self._t_end - self._t_start)

For more details about context managers, please refer elsewhere. Let's use our timer to time the kernel:

In [ ]:
# Set up the host vectors
N = 200*1000*1000
x = rng.uniform(size=N)
y = rng.uniform(size=N)

# Copy and allocate data on the device
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_z = cuda.device_array_like(x)

# Set up the kernel invocation
block_size = 128
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

# Call the kernel
with time_region() as t_kernel:
    _vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)

print(f'CUDA kernel time: {t_kernel.elapsed_time()} s')

# Copy back the result to the host
res = d_z.copy_to_host()

# Validate the result
assert np.allclose(x + y, res)

Our timer seems to work fine; we still measure the kernel launch time as with `%timeit`. In order to measure the actual kernel execution time, we need to block the CPU calling thread until the kernel finishes, immediately after we launch the kernel. We can achieve that with `cuda.synchronize()`:

In [ ]:
# Set up the host vectors
N = 200*1000*1000
x = rng.uniform(size=N)
y = rng.uniform(size=N)
z = np.empty_like(x)

# Copy and allocate data on the device
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_z = cuda.device_array_like(x)

# Set up the kernel invocation
block_size = 128
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

# Call the kernel
with time_region() as t_kernel:
    _vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)
    cuda.synchronize()

with time_region() as t_ref:
    np.add(x, y, out=z)

print(f'CUDA kernel time: {t_kernel.elapsed_time()} s')
print(f'Numpy time:       {t_ref.elapsed_time()} s')


# Copy back the result to the host
res = d_z.copy_to_host()

# Validate the result
assert np.allclose(x + y, res)

Not bad, the CUDA kernel is more than 100x faster than the numba vectorized implementation.

Before analysing how good or bad this is, let's see an alternative way for measuring the kernel time that actually avoids the use of `cuda.synchronize()`.

## Measuring the kernel execution time with CUDA events

Inserting `cuda.synchronize()` without a reason could slow down your application, since it not only blocks the current CPU thread, but also imposes a synchronization point for all the CUDA streams on the GPU that are currently running in parallel.

> A CUDA stream is essentially a series of sequential operations (data transfers, kernel launches, etc.) that execute on the GPU. Multiple CUDA streams may run independently on the GPU, thus allowing overlapping of operations, such as data transfers and execution of kernels.

To avoid this, but also to obtain a more precise measurement, you can use [CUDA events](http://docs.nvidia.com/cuda/cuda-c-programming-guide/#events). You can imagine CUDA events as milestones associated with timestamps that you can insert between operations in a CUDA stream. Let's how we can adapt our `time_region` context manager to use CUDA events:

In [ ]:
class time_region_cuda:
    def __init__(self, time_offset=0, cuda_stream=0):
        self._t_start = cuda.event(timing=True)
        self._t_end = cuda.event(timing=True)
        self._time_off = time_offset
        self._cuda_stream = cuda_stream

    def __enter__(self):
        self._t_start.record(self._cuda_stream)
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self._t_end.record(self._cuda_stream)
        self._t_end.synchronize()

    def elapsed_time(self):
        return self._time_off + 1.e-3*cuda.event_elapsed_time(self._t_start, self._t_end)

To measure a data region with CUDA events you first need to create two events: one for the start and one for the end. You can achieve that with the `cuda.event(timing=True)`. To start counting, you need to call `record()` on the starting event marking the "arrival" to that milestone. Similarly, you call `record()` on the ending event to mark the end of the region. Then you can obtain the elapsed time using the corresponding function as shown in the example above.

Let's rewrite our vector addition example using the CUDA event timers:

In [ ]:
# Set up the host vectors
N = 200*1000*1000
x = rng.uniform(size=N)
y = rng.uniform(size=N)
z = np.empty_like(x)

# Copy and allocate data on the device
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_z = cuda.to_device(z)

# Set up the kernel invocation
block_size = 128
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

# Call the kernel
with time_region_cuda() as t_kernel:
    _vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)

with time_region() as t_ref:
    np.add(x, y, out=z)
    
with time_region_cuda() as t_copyout:
    d_z.copy_to_host(z)
        
print(f'CUDA kernel time: {t_kernel.elapsed_time()} s')
print(f'Numpy time:       {t_ref.elapsed_time()} s')


# Copy back the result to the host
# Validate the result
assert np.allclose(x + y, z)

As we can see the execution time obtained is the correct one without having to use `cuda.synchronize()`.

## Assessing the performance of the kernel

The question that arises is how good is the performance that we achieve. Let's inspect further the kernel. Each thread does two `float64` reads from the memory and one write and performs an addition. That means for one floating operation, the kernel must transfer to/from memory 24 bytes from the main memory. This gives us an  *arithmetic intensity* or *flop:byte ratio* of 0.0417. The lower this ratio is for a computational kernel, the more likely will be that the kernel is memory bandwidth bound. As the ratio increases, the kernel tends to be more compute bound. The theory behind the arithmetic intensity is covered by the *Roofline* performance model, which is outside the scope of this tutorial. For the moment, let's compute two performance metrics, the `Gflop/s` achieved by the kernel and the data transfer rate to/from memory:

In [ ]:
print(f'Performance: {1.e-9*N/t_kernel.elapsed_time()} Gflop/s')
print(f'Transfer rate: {1.e-9*3*N*8/t_kernel.elapsed_time()} GB/s')

#### And for NumPy

In [ ]:
print(f'Performance: {1.e-9*N/t_ref.elapsed_time()} Gflop/s')
print(f'Transfer rate: {1.e-9*3*N*8/t_ref.elapsed_time()} GB/s')

##### As you can, the GPU can deliver more than 100x bandwidth compared to the CPU. Checking the [datasheet](https://resources.nvidia.com/en-us-data-center-overview-mc/en-us-data-center-overview/grace-hopper-superchip-datasheet-partner) of the NVIDIA GH200 superchip, we can see that the peak nominal memory bandwidth is 4TB/s, meaning that our kernel utilizes 75-80% of the peak bandwidth.

* Achieving the nominal peak memory bandwidth is usually not possible with real-life computational kernels, even with very low arithmetic intensity.  

* <mark>NOTE</mark>: The numpy vector addition performance is not ideal, since it can't reach the memory bandwidth limit as it ought to. The problem could be related to CPU affinity issues, but we are not going to address them in this tutorial.For the Grace CPU, the effective memory bandwidth is ~500 GB/s.

## Understanding the data transfer overhead

So far we have only focused on the performance of the kernel. There is still a quite important topic we have not yet addressed. CUDA kernels require that the data they operate on is located on the device and we need to move that data there from the host. What is the cost of this data movement? Let's time our benchmark code:

In [ ]:
# Set up the host vectors
N = 200*1000*1000
x = rng.uniform(size=N)
y = rng.uniform(size=N)


# Copy and allocate data on the device
with time_region_cuda() as t_copyin:
    d_x = cuda.to_device(x)
    d_y = cuda.to_device(y)

with time_region_cuda() as t_create:
    d_z = cuda.device_array_like(x)

# Set up the kernel invocation
block_size = 128
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

# Call the kernel
with time_region_cuda() as t_kernel:
    _vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)

with time_region() as t_ref:
    z = x + y

print(f'CUDA kernel time: {t_kernel.elapsed_time()} s')
print(f'Numpy time:       {t_ref.elapsed_time()} s')


# Copy back the result to the host
with time_region_cuda() as t_copyout:
    d_z.copy_to_host(res)

print(f'Copyin time:  {t_copyin.elapsed_time()} s')  
print(f'Create time:  {t_create.elapsed_time()} s')    
print(f'Copyout time: {t_copyout.elapsed_time()} s')    


# Validate the result
assert np.allclose(x + y, res)

#### The data copy times are quite important! Data transfers is the No. 1 optimization that you should do when programming for the GPUs. You must minimize the data transfers to/from GPU by keeping the necessary data on the GPU for as long as it is needed.

Before closing this discussion, let's see how fast is the data moved over to the GPU:

In [ ]:
print(f'Copyin rate: {1e-9*2*N*8/t_copyin.elapsed_time()} GB/s')

This is bound by the data rate of the [NVLink-C2C](https://www.nvidia.com/en-us/data-center/nvlink-c2c/) connecting the CPU/GPU of the Grace Hopper superchip.


In [ ]:
print(f'Copyout rate: {1e-9*N*8/t_copyout.elapsed_time()} GB/s')

## Profiling the CUDA code (optional)

In this simple example of vector addition we assessed the performance and identified the bottlenecks ourselves, by analyzing the code structure and reasoning about it. In more complex codes or codes that you are not very familiar with, it would be good if this analysis could be done by a dedicated tool. Not to be misunderstood, understanding the code structure and its memory and compute requirements is essential for optimizing it in any case, but using a *performance profiler* is very handy for analyzing the performance bottlenecks, for helping you prioritizing your optimization targets and for understanding how much room for improvement exists.

NVIDIA provides [Nsight Systems](https://developer.nvidia.com/nsight-systems) for profiling the code and inspecting the results.

The `numba-cuda/src/vecadd.py` file contains the vector addition example as we have finally presented it here. Let's do a basic profing.

First, we need to open a new terminal ("File->New->Terminal") and activate the virtual environment of the course's Python kernel.

Now it's time to do our basic profiling:

```bash
cd numba-cuda/src
nsys profile -o vecadd.nsys-rep python3 vecadd.py $((200*1000*1000))
```